In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from glob import glob
import pydicom
import tensorflow as tf
import tqdm as tqdm
import tensorflow_io as tfio
import pathlib

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [1]:
!pip install -q ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl
!pip install -q ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q ../input/for-pydicom/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

# Creating the Dataset tf.Dataset.from_tensor_slices #

In [ ]:
class cl_CreatingDataset:
    def __init__(self, imageHeight, imageWidth, batch_size):
        self.imageHeight = imageHeight
        self.imageWidth = imageWidth
        self.batch_size = batch_size

    
    def creatingPathList(self, trainImagesPath, trainDfPath):
        trainDf = pd.read_csv(trainDfPath)
        trainImagesPathList = []
        labels = []
        # reading the tain CSV
        trainDf = pd.read_csv(trainDfPath)
        for i in tqdm.tqdm(range(len(trainDf))):
            folderName = trainDf["StudyInstanceUID"].iloc[i]
            imageFolderPath = os.path.join(trainImagesPath, folderName)

            for file in glob(os.path.join(imageFolderPath, "*.dcm")):
                # taking the imageName
                trainImagesPathList.append(file)
                # creating the labels
                label = np.array([trainDf["C1"].iloc[i],trainDf["C2"].iloc[i], trainDf["C3"].iloc[i], trainDf["C4"].iloc[i], trainDf["C5"].iloc[i], 
                                 trainDf["C6"].iloc[i], trainDf["C7"].iloc[i], trainDf["patient_overall"].iloc[i]])
                labels.append(label)
        
        return trainImagesPathList, labels
    
    def parse_function(self, filename, label):
        image_bytes = tf.io.read_file(filename)
        image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.float32)
        #image = tf.image.convert_image_dtype(image, tf.float32)
        resized_image = tf.image.resize(image, [self.imageHeight, self.imageWidth])
        compressedImage = tf.squeeze(resized_image, axis = 0)
        finalImage = tf.repeat(compressedImage, repeats=[3], axis = 2)
        return finalImage, label
        
    def train_preprocess(self, image, label):
        return image, label


    def creatingDataset(self, filenames, labels):
        dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
        dataset = dataset.shuffle(len(filenames))
        dataset = dataset.map(self.parse_function, num_parallel_calls=4)
        dataset = dataset.map(self.train_preprocess, num_parallel_calls=4)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.prefetch(1)

        return dataset

# Creating Dataset using Generator Concept (tf.data.Dataset.fromgenerator) #

In [3]:
DATA_DIR = "../input/rsna-2022-cervical-spine-fracture-detection/"
trainCsv = "../input/rsna-2022-cervical-spine-fracture-detection/train.csv"
train_df = pd.read_csv(trainCsv)
print(train_df.head())

            StudyInstanceUID  patient_overall  C1  C2  C3  C4  C5  C6  C7
0   1.2.826.0.1.3680043.6200                1   1   1   0   0   0   0   0
1  1.2.826.0.1.3680043.27262                1   0   1   0   0   0   0   0
2  1.2.826.0.1.3680043.21561                1   0   1   0   0   0   0   0
3  1.2.826.0.1.3680043.12351                0   0   0   0   0   0   0   0
4   1.2.826.0.1.3680043.1363                1   0   0   0   0   1   0   0


In [4]:
def load_dicom(path):
    """
    reads a dicom file and loads the image array inside it
    inputs:
        path: the path of the required dicom file
    returns:
        data: image pixel arrays
    """
    img=pydicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [5]:
def data_generator():
    for i, study_instance in enumerate(train_df.StudyInstanceUID[:5]):
        for dcm in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            train_labels = []
            path = DATA_DIR + f"train_images/{study_instance}/{dcm}"
            print(path)
            img = load_dicom(path)
            
            # resize each image into a shape of (512, 512)
            img = np.resize(img, (512, 512))
            #  normalize image
            img = img / 255.0
            # convert from gray scale to rgb, this will be helpful incase we want to use pretrained models
            img = tf.expand_dims(img, axis=-1)
            img = tf.image.grayscale_to_rgb(img)
            
            train_labels.extend([
                train_df.loc[i, "C1"],
                train_df.loc[i, "C2"],
                train_df.loc[i, "C3"],
                train_df.loc[i, "C4"],
                train_df.loc[i, "C5"],
                train_df.loc[i, "C6"],
                train_df.loc[i, "C7"],
                train_df.loc[i, "patient_overall"] # end with patient overall
            ])
            yield img, train_labels

### Testing Data after generation ###

In [6]:
def testDataset(dataset):
    for img, label in dataset.take(1):
        print(img.shape)
        print(label.shape)
        print(label)

## Splitting the dataset ##

In [7]:
def splitDataset(dataset, trainFactor, img_count): # here it refers to tf.dataset
    train_dataset = dataset.take(int(trainFactor * img_count))
    validation_dataset = dataset.take(int((1 - trainFactor)* img_count))
    
    return train_dataset, validation_dataset
    

In [8]:
def configure_for_performance(data):
    data = data.cache()
#   data = data.shuffle(buffer_size=300)
    data = data.batch(4)
    data = data.prefetch(buffer_size=tf.data.AUTOTUNE)
    return data

## Creating the model ##

In [13]:
def creating_model(opt):
    IMG_SHAPE = (512, 512, 3)
    base_model = tf.keras.applications.EfficientNetB5(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    
    base_model.trainable = False
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(8, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    #model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten

# Define Alex Net model
def alex_net():
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(512,512,3), kernel_size=(11,11),\
     strides=(4,4), padding='valid', activation="relu"))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', activation="relu"))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

   # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(512*512*3,), activation="relu"))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096, activation="relu"))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000, activation="relu"))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer with 8 probability classes
    model.add(Dense(8, activation="softmax"))
    return model

trainCsv = "../input/rsna-2022-cervical-spine-fracture-detection/train.csv"
trainImagePath = "../input/rsna-2022-cervical-spine-fracture-detection/train_images"
creDataset = cl_CreatingDataset(224, 224, 4)
trainImagesPathList, labels = creDataset.creatingPathList(trainImagePath, trainCsv)
print("train Image list :", len(trainImagePath), ".....", "train label list : ", len(labels))


In [10]:
# this is required because of length of dataset is not valid for generators
def getImageCount():
    img_count = 0
    for _, study_instance in enumerate(train_df.StudyInstanceUID[:5]):
        for _ in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            img_count += 1
            
    return img_count

In [14]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# creating the dataset
dataset = tf.data.Dataset.from_generator(data_generator, (tf.float32, tf.int8))
for img, label in dataset.take(1):
    print(img.shape)
    print(label.shape)
    print(label)

# splitting the dataset
trainFactor = 0.8
img_count = getImageCount()
print("[***] Images for training and validation : ", img_count)

train_data, validation_data = splitDataset(dataset, trainFactor, img_count)
train_dataset = configure_for_performance(train_data)
validation_dataset = configure_for_performance(validation_data)
#print("[*]------Length of Trainig data : ", len(train_dataset))
#print("[*]------Length of validation data : ", len(validation_dataset))

# creating and compiling the model
INIT_LR = 1e-3
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)
model = creating_model(opt)
#model = alex_net()
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
# training
EPOCHS = 10
BATCH_SIZE = 4
#model.fit(train_dataset, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = validation_dataset)


../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/240.dcm
(512, 512, 3)
(8,)
tf.Tensor([1 1 0 0 0 0 0 1], shape=(8,), dtype=int8)
[***] Images for training and validation :  1734
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
efficientnetb5 (Functional)  (None, 16, 16, 2048)      28513527  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 16392     
Total params: 28,529,919
Trainable para

In [15]:
model.fit(train_dataset, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = validation_dataset)

Epoch 1/10
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/240.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/12.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/210.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/120.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/208.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/141.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/18.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/9.dcm
      1/Unknown - 13s 13s/step - loss: 6.3229 - categorical_accuracy: 0.0000e+00../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/97.dcm
../input/rsna-2022-cerv

KeyboardInterrupt: 

../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/393.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/245.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/39.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/45.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/14.dcm
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262/231.dcm


In [ ]:
for image, label in train_dataset.take(1):
    imageNp = image.numpy()
    print(imageNp.shape)

In [ ]:
trainCsv = "../input/rsna-2022-cervical-spine-fracture-detection/train.csv"
train_df = pd.read_csv(trainCsv)
DATA_DIR = "../input/rsna-2022-cervical-spine-fracture-detection/"
for i, study_instance in enumerate(train_df.StudyInstanceUID[:5]):
    for dcm in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            path = DATA_DIR + f"train_images/{study_instance}/{dcm}"
            print(path)